In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/d10800_BS1_JoinedTrans.csv")
df.head()

,src_file,row_id,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,13,0,0,0,3,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,19,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,21,0,0,0,0,0,3,0,0,...,0,0,0,0,3,0,0,3,0,0
4,BS1__BC__1-OK-B_cells_Bmemory-580-BOS-BROADS1-...,26,0,0,0,0,0,3,0,0,...,3,0,0,0,0,0,0,0,0,0


In [4]:
X = df.drop(columns=['src_file', 'row_id'])

In [5]:
X.shape

(13183, 10800)

In [6]:
y = pd.array(df['src_file'])

In [7]:
from sklearn.feature_selection import VarianceThreshold

In [8]:
selector = VarianceThreshold()
selector.fit(X)
X_selected = X[X.columns[selector.get_support(indices=True)]]
X_selected.head()

,hg38_ASF1A,hg38_CCT7,hg38_EPOR,hg38_NECAP2,hg38_LRRC8A,hg38_NDUFS8,hg38_MRPL22,hg38_HIST1H2BC,hg38_LY96,hg38_UBE2E2-AS1,...,hg38_TBC1D10C,hg38_TMEM128,hg38_ASPSCR1,hg38_CDC20,hg38_DNAJB9,hg38_OSTM1,hg38_LCNL1,hg38_PHB,hg38_LRRC29,hg38_ZFP36
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,3,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,2,0,3
3,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,3,0,0,3,0,0
4,0,0,0,0,0,3,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0


In [9]:
groups = []
for s in y:
    src_file = s.split('_')
    group = src_file[2].replace('C', '')
    groups.append(group)
groups = pd.array(groups)
y = pd.DataFrame({'Group': groups})

In [10]:
def load_data(data, columns):
    df = pd.read_csv(data)
    X = df[columns]
    y = pd.array(df['src_file'])
    groups = []
    for s in y:
        src_file = s.split('_')
        group = src_file[2].replace('C', '')
        groups.append(group)
    groups = pd.array(groups)
    y = pd.DataFrame({'Group': groups})
    print(y['Group'].unique())
    return X, y

In [11]:
data = pd.concat([X_selected, y], axis=1)
data.to_csv('../data/preprocessed_data_BS1.csv', index=False)

In [12]:
X, y = load_data(data="../data/d10800_BS2_JoinedTrans.csv", columns=X_selected.columns)

<StringArray>
['B', 'D', 'M', 'NK', 'T']
Length: 5, dtype: string


In [13]:
data = pd.concat([X, y], axis=1)
data.to_csv('../data/preprocessed_data_BS2.csv', index=False)

In [14]:
X, y = load_data(data="../data/d10800_10x_JoinedTrans.csv", columns=X_selected.columns)

<StringArray>
['B', 'M', 'NK', 'T']
Length: 4, dtype: string


In [15]:
data = pd.concat([X, y], axis=1)
data.to_csv('../data/preprocessed_data_10x.csv', index=False)

In [16]:
X, y = load_data(data="../data/d10800_GEO_JoinedTrans.csv", columns=X_selected.columns)

<StringArray>
['B', 'M', 'NK', 'T']
Length: 4, dtype: string


In [17]:
data = pd.concat([X, y], axis=1)
data.to_csv('../data/preprocessed_data_GEO.csv', index=False)